# **Investment Portfolio Management**

## **About the scenario**
This scenario demonstrates a common project, where notebooks are used to orchestrate complex tasks involving real-time data fetching, computation, and report delivery. The notebook is modular, and secure, employing batch processing, error handling, and proper separation of concerns.

In this scenario, we will:

1. Upload a CSV file containing the user’s investment portfolio to the Azure OpenAI Service.
2. Fetch real-time stock prices via the Yahoo! Finanace API utilizing *Function Calling*.
3. Perform calculations on the portfolio using *Code Interpreter*.

## **Time**
You should expect to spend 10-15 minutes building and running this scenario. 

## **Before you begin**

#### Step 1: Install required libraries
Install dependencies directly within a Jupyter notebook is a good practice because it ensures that all required packages are installed in the correct versions, making the notebook self-contained and reproducible. This approach helps other users or collaborators to set up the environment quickly and avoid potential issues related to missing or incompatible packages.

In [ ]:
# Install the packages
%pip install -r ./requirements.txt

#### Step 2: Setting up the environment

Before we begin, we'll load all necessary environment variables from a `.env` file. These variables contain sensitive information such as API keys and endpoint URLs. Ensure your `.env` file is properly configured in the `.venv/.env` format. In order to run the following code, the `.env` file must contain the follow secrets:

- AZURE_OPENAI_API_KEY
- AZURE_OPENAI_ENDPOINT
- AZURE_OPENAI_DEPLOYMENT
- AZURE_OPENAI_API_VERSION 

For more information about leveraging Python Virtual Environments can be found [here]().

In [94]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(dotenv_path=".venv/.env")

# Retrieve the secrets
__AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
__AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
__AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
__AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# Verify environment variables
if not all([ __AZURE_OPENAI_API_KEY,
             __AZURE_OPENAI_ENDPOINT, 
             __AZURE_OPENAI_DEPLOYMENT, 
             __AZURE_OPENAI_API_VERSION]):
    raise EnvironmentError("One or more environment variables are missing. Please check the .env file.")
else:
    print("Environment variables loaded successfully.")

Environment variables loaded successfully.


## **Azure OpenAI setup**

#### Step 1: Initializing the Azure OpenAI Client

Next, we’ll initialize the Azure OpenAI client. This client allows us to interact with Azure OpenAI's service and file management APIs. You’ll need your API key, endpoint, and API version, which we just loaded from the environment.

In [95]:
from openai import AzureOpenAI, OpenAIError

# Initialize the AzureOpenAI client
try:
    openai_client = AzureOpenAI(
        api_key=__AZURE_OPENAI_API_KEY,
        api_version=__AZURE_OPENAI_API_VERSION,
        azure_endpoint=__AZURE_OPENAI_ENDPOINT
    )
    print("AzureOpenAI client initialized.")
except OpenAIError as e:
    raise ConnectionError(f"Failed to initialize AzureOpenAI client: {e}")


AzureOpenAI client initialized.


#### Step 2: Upload supporting file to Azure OpenAI deployment

Now, we'll upload the `investment_portfolio.csv` file from the `\data` directory to Azure OpenAI, ensuring any existing `investment_portfolio.csv` removed beforehand to ensure the latest version of the file is used and no duplicates exist. This process will handle the entire upload. The file is necessary for this scenario, but its contents can be modified as long as the file structure remains unchanged. The file schema must be as follows:

- Symbol
- Average_Cost
- QTY

In [96]:
# Directory containing files to upload
directory="data"
portfolio_file="investment_portfolio.csv"

# Check if the directory exists
if not os.path.isdir(directory):
    print(f"Directory '{directory}' does not exist.")
    raise FileNotFoundError(f"Directory '{directory}' does not exist.")

file_path = os.path.join(directory, portfolio_file)

# Check if the file exists
if not os.path.isfile(file_path):
    print(f"Skipping non-file item: {portfolio_file}")

try:
    # Delete existing file in Azure OpenAI Service if it has the same name and purpose
    existing_files = openai_client.files.list()
    for f in existing_files:
        if f.filename == portfolio_file and f.purpose == "assistants":
            openai_client.files.delete(file_id=f.id)
            print(f"Deleted existing file: {portfolio_file}")

    # Upload new file
    with open(file_path, "rb") as file_data:
       portfolio_file = openai_client.files.create(file=file_data, purpose="assistants")
    print(f"Uploaded file: {portfolio_file.filename}")

except OpenAIError as e:
    print(f"Error processing file '{portfolio_file}': {e}")
except Exception as e:
    print(f"Unexpected error with file '{portfolio_file}': {e}")

Deleted existing file: investment_portfolio.csv
Uploaded file: investment_portfolio.csv


## **Azure OpenAI Assistant**

### Step 1: Define function for Assistant
The `fetch_stock_price` function retrieves the stock data for a specified `ticker symbol` using the `yfinance` library, specifically pulling the latest data for the last trading day.

In [97]:
import yfinance as yf

def fetch_stock_price(ticker_symbol: str) -> str:
    """
    Fetch the latest stock price for a given ticker symbol.

    Parameters:
    - ticker_symbol (str): The ticker symbol of the stock to retrieve data for.

    Returns:
    - str: The closing price of the stock for the latest trading day, or an error message if data is unavailable.

    Example:
    >>> fetch_stock_price("AAPL")
    "148.9"
    """
    
    try:
        # Fetch the stock's trading history for the last day
        stock = yf.Ticker(ticker_symbol)
        stock_data = stock.history(period="1d")

        # Check if the data is empty, indicating an invalid ticker or no data available
        if stock_data.empty:
            return f"Error: No data found for ticker symbol: {ticker_symbol}"

        # Retrieve and return the latest closing price
        latest_close_price = stock_data['Close'].iloc[-1]
        return str(round(latest_close_price, 3))

    except KeyError as e:
        return f"Error: Data missing for key: {e}. Verify the ticker symbol."

    except Exception as e:
        return f"Error: Unexpected issue occurred - {type(e).__name__}: {e}"
    
print("Function defined successfully.")

Function defined successfully.


### Step 2: Define Function Calling tool for Assistant
The tool-calling definition informs the LLM about available tools. For our scenario, it enables code interpretation and function calling. The `fetch_stock_price` function configuration is to be used along side the code interpreter tool. By adding this tool to the Assistant, we enable it to retrieve and interpret stock data when a user prompts it with related queries.

In [98]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "fetch_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            "parameters": {
                "type": "object",
                "properties": {"ticker_symbol": {"type": "string", "description": "The ticker symbol of the stock"}},
                "required": ["ticker_symbol"],
            },
        },
    }
]

print("Tools list defined successfully.")

Tools list defined successfully.


### Step 3: Creating the Investment Management Assistant
In this step, we create an assistant equipped with specialized tools, including a code interpreter, to handle investment-related queries and utilize the previously defined function calls. This assistant will analyze the uploaded portfolio file and offer valuable insights.

In [99]:
# Create the assistant with code interpreter and function calling tools enabled
try:
    assistant = openai_client.beta.assistants.create(
        name="Investment Management Assistant",
        instructions=(
            "You are an expert investment analyst. "
            "Use your knowledge base to answer questions about personal investment portfolio management."
        ),
        model=__AZURE_OPENAI_DEPLOYMENT,
        tools=tools_list
    )
    print("Assistant created successfully.\n", assistant)
except OpenAIError as e:
    print("Error creating assistant:", e)

Assistant created successfully.
 Assistant(id='asst_xvxLgxK8MUMutkrrMp5IMR92', created_at=1731700777, description=None, instructions='You are an expert investment analyst. Use your knowledge base to answer questions about personal investment portfolio management.', metadata={}, model='gpt-4o', name='Investment Management Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter'), FunctionTool(function=FunctionDefinition(name='fetch_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol.', parameters={'type': 'object', 'properties': {'ticker_symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['ticker_symbol']}, strict=False), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)


## **Querying the Assistant**

### Step 1: Starting a New Conversation Thread
Let's create a new thread to handle user interactions. Each thread allows for a dedicated conversation with the assistant.

In [100]:
# Create a conversation thread
try:
    thread = openai_client.beta.threads.create()
    print("Thread created successfully.\n", thread)
except OpenAIError as e:
    print("Error creating thread:", e)

Thread created successfully.
 Thread(id='thread_aDZQEiy8QK3QwMaHk3JiM6TI', created_at=1731700780, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


### Step 2: Adding User Message
In this step, we add a question to the thread. For this demonstration, we'll what the latest closing price is for specified company (*Microsoft*) that will leverage *function calling* to retrieve this information from the `fetch_stock_price` function. Then, code interpreter will use this information as well as the `QTY` data from the `investment_portfolio.csv` that was uploaded to calculate the **total investment**.

In [ ]:
# Define the user question
prompt_content = "What is the latest closing price for Microsoft? What is my total investment for MSFT as of today?"

# Add the question to the thread
try:
    message = openai_client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt_content,
        attachments=[  # Add files by using the attachments parameter
            {"file_id": portfolio_file.id, "tools": [{"type": "code_interpreter"}]}
        ],
    )
    print("User question added:", message)
except OpenAIError as e:
    print("Error adding user question:", e)


### Step 3: Running the Assistant
Now that the assistant and thread are set up, we'll initiate the assistant's response process. This will analyze the user prompt and provide insights based on the investment portfolio data.


In [ ]:
# Initiate the assistant's response
try:
    run = openai_client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions=prompt_content,
    )
    print("Run started:", run)
except OpenAIError as e:
    print("Error starting run:", e)

 <code style="background:yellow;color:black">Define the dictionary `available_functions` that maps function names to their corresponding implementations, in this case, fetch_stock_price</code>

In [ ]:
available_functions = {"fetch_stock_price": fetch_stock_price}

### Step 4: Monitor run status
 <code style="background:yellow;color:black">The assistant may take some time to analyze and respond so we must monitor the run status. Each status will perform an action. To understand the status actions more, lets simulate a loops. Click the next 4 cells multiple times to see the progress of the Assitant Run.


Generally we use a loop to check the status and perform actions based on the outcome of each check. 
Retrieves the current status of the OpenAI Assistant run. The status is printed in a formatted JSON string for clarity. Depending on the status of the run, *different actions are taken*. </code>

*Status: Queued*
<code style="background:yellow;color:black">What does this mean - probably wont use. but explain</code>

A Run is an instance where an Assistant operates within a Thread. During a Run, the Assistant processes the Thread's Messages and its configuration to perform tasks using models and tools, adding additional Messages to the Thread as part of its operations.

In [ ]:
import json

# Retrieve the run status
run = openai_client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
#print(run.model_dump_json(indent=4))

if run.status in ('queued', 'in_progress'):
    print(f"Run this cell again to monitor the status.\nCurrent Status: {run.status}")
else:
    print(f"Monitoring the run status...\nCurrent Status: {run.status}")

#### Step 4a: Failed status

If the status is `failed` we will print the error message with relevant information to aid in troubleshooting.

In [ ]:
#print(run.model_dump_json(indent=4))

if run.status == "failed":
    print("Assistant run failed. Please try again.")
else:
    print(f"Assistant run has not failed...\nNavigate to and execute the '{run.status}' cell.")


#### Step 4b: Requires Action status

If the status is `requires_action`, first we need to check if the required action is to submit tool outputs and iterate over the tool calls, ensuring that the requested function exists in the available_functions dictionary. If the function exists, it is called with the provided arguments, and the response is stored. After processing all tool calls, we submit the tool outputs back to the OpenAI client.

In [ ]:
if run.status == "requires_action":
    print("Function Calling ...")
    tool_responses = []
    if (
        run.required_action.type == "submit_tool_outputs"
        and run.required_action.submit_tool_outputs.tool_calls is not None
    ):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls

        for call in tool_calls:
            if call.type == "function":
                if call.function.name not in available_functions:
                    raise Exception("Function requested by the model does not exist")
                function_to_call = available_functions[call.function.name]
                tool_response = function_to_call(**json.loads(call.function.arguments))
                tool_responses.append({"tool_call_id": call.id, "output": tool_response})
                print(f"Function '{call.function.name}' called successfully. \nOutput: {tool_response}\n")

    run = openai_client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id, run_id=run.id, tool_outputs=tool_responses
    )

    print(f"Results submitted successfully. Go back to the first cell, 'Monitor Run Status' and execute again.")
else:
    print(f"Navigate to and execute the {run.status} cell.")

#### Step 4c: Completed status

If the status is `completed`, we will fetch and print all messages in the thread, displaying the role and content of each message.The messages are printed in reverse order because messages in a thread are in FILO (First-In-Last-Out) order and in order to make the messages more conversational for ease of user reading, we must reverse the order.

In [ ]:
if run.status == "completed":
    messages = openai_client.beta.threads.messages.list(thread_id=thread.id, order="asc", after=message.id) # Ascending order for output

    print(f'Run completed!\n\nMESSAGES\n')

    # Loop through messages and print content based on role
    for msg in messages.data:
        role = msg.role
        content = msg.content[0].text.value
        print(f"{role.capitalize()}: {content}")

else:
    print(f"Navigate to and execute the {run.status} cell.")



#### Next Step / Let's refine 
Wrap in function - explain the 

We'll use a polling loop to periodically check the assistant's status and retrieve the answer once it's available.

In [101]:
import time
import json

available_functions = {"fetch_stock_price": fetch_stock_price}

def process_message(thread_id, prompt_message, attachments_list=None):
    try:
        # Add the prompt to the thread
        message = openai_client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=prompt_message,
            attachments=attachments_list,
        )
        print("User message added...") #:", message)

        # Initiate the assistant's response
        run = openai_client.beta.threads.runs.create(
            thread_id=thread_id,
            assistant_id=assistant.id,
            instructions=prompt_message,
        )
        print("Run started...") #:", run)
    except OpenAIError as e:
        print("Error starting run:", e)

    while True:  # Polling to monitor Run status
        time.sleep(5)  # Wait 5 seconds to give the process time to move past `queued` state

        # Retrieves the thread’s response.
        run = openai_client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        run_status = run.status
        print(f"Run Status: {run_status}\n")
        if run_status == 'completed':
            # Get all messages in thread to read
            thread_messages = openai_client.beta.threads.messages.list(thread_id=thread_id, order="asc") #, after=message.id)

            # Loop through thread messages and print content
            for thread_message in thread_messages.data:
                role = thread_message.role
                content = None
                if isinstance(thread_message.content, list) and thread_message.content:
                    first_content = thread_message.content[0]
                    if hasattr(first_content, 'text') and hasattr(first_content.text, 'value'):
                        content = first_content.text.value
                    else:
                        content = str(first_content)
                else:
                    content = str(thread_message.content)

                print(f"{role.capitalize()}: {content}")
            
            break
        elif run.status == "failed":
            messages = openai_client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt_message}\nAssistant:\n{answer}\n")

            # Handle failed
            break

        elif run.status == "requires_action" and run.required_action.type == "submit_tool_outputs":
            print("Function calling initiated...")
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_responses = []
            
            # Iterate over each function call requested by the assistant
            for call in tool_calls:
                # Check if the call is a function and if the function exists is our custom function
                if call.type == "function" and call.function.name in available_functions:
                    func = available_functions[call.function.name]  # Retrieve the function reference
                    
                    # Parse the function arguments from JSON and execute the function
                    tool_response = func(**json.loads(call.function.arguments))
                    
                    # Store the tool call ID and output to later send back to the assistant
                    tool_responses.append({"tool_call_id": call.id, "output": tool_response})
                    print(f"Executed '{call.function.name}'. Output: {tool_response}")
                
                else:
                    # Raise an error if the function is not in available_functions to handle unexpected requests
                    raise ValueError(f"Requested function '{call.function.name}' is not available.")
            
            # Submit all collected tool outputs back to the assistant to satisfy the required action
            run = openai_client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id, 
                run_id=run.id, 
                tool_outputs=tool_responses
            )
            print("Function call(s) completed successfully.")

        else:
            time.sleep(5)

print("Function `process_message` created successfully.")


Function `process_message` created successfully.


In [102]:
process_message(thread_id=thread.id, prompt_message="What is today's date?")

User message added...
Run started...
Run Status: completed

User: What is today's date?
Assistant: Today's date is November 15, 2024.


In [103]:
attachments=[  # Add files by using the attachments parameter
            {"file_id": portfolio_file.id, "tools": [{"type": "code_interpreter"}]}
        ]

process_message(thread_id=thread.id, prompt_message="What stock do I have the most investment in?", attachments_list=attachments)

User message added...
Run started...
Run Status: in_progress

Run Status: completed

User: What is today's date?
Assistant: Today's date is November 15, 2024.
User: What stock do I have the most investment in?
Assistant: The file contains the following columns: `Symbol`, `Average_Cost`, and `QTY`. These columns represent the ticker symbol of the stock, the average cost per share, and the quantity of shares held, respectively.

To determine which stock you have the most investment in, I will calculate the total investment for each stock (Average Cost * Quantity) and identify the one with the highest value.
Assistant: The stock you have the most investment in is Tesla (TSLA) with a total investment of $90,000.


In [104]:
process_message(thread_id=thread.id, prompt_message="What is the current stock price for Amazon?")

User message added...
Run started...
Run Status: requires_action

Function calling initiated...
Executed 'fetch_stock_price'. Output: 200.65
Function call(s) completed successfully.
Run Status: completed

User: What is today's date?
Assistant: Today's date is November 15, 2024.
User: What stock do I have the most investment in?
Assistant: The file contains the following columns: `Symbol`, `Average_Cost`, and `QTY`. These columns represent the ticker symbol of the stock, the average cost per share, and the quantity of shares held, respectively.

To determine which stock you have the most investment in, I will calculate the total investment for each stock (Average Cost * Quantity) and identify the one with the highest value.
Assistant: The stock you have the most investment in is Tesla (TSLA) with a total investment of $90,000.
User: What is the current stock price for Amazon?
Assistant: The current stock price for Amazon (AMZN) is $200.65.


In [ ]:
process_message(thread_id=thread.id, prompt_message="Show a pie chart of my investments?")

In [ ]:
import time

def process_message_old(thread_id, prompt_message, attachments_list=None):

    try:
        # Add the prompt to the thread
        message = openai_client.beta.threads.messages.create(
            thread_id=thread_id,
            role="user",
            content=prompt_message,
            attachments=attachments_list,
        )
        print("User question added:", message)

        # Initiate the assistant's response
        run = openai_client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id,
            instructions=prompt_message,
        )
        print("Run started:", run)
    except OpenAIError as e:
        print("Error starting run:", e)


    while True: # Polling to monitor Run status
        
        # Wait 5 seconds to give the process time to move past `queued` state
        time.sleep(5)

        # Retrieves the thread’s response.
        run = openai_client.beta.threads.runs.retrieve(
            thread_id=thread_id,
            run_id=run.id
        )

        run_status = run.status
        print(f"Run Status: {run_status}\n")
        if run_status =='completed':

            # Get all messages in thread to read
            thread_messages = openai_client.beta.threads.messages.list(thread_id=thread_id, order="asc") #, after=message.id)

            # Loop through thread messages and print content
            for thread_message in thread_messages.data:
                role = thread_message.role
                #content = thread_message.content[0].text.value
                
                # Safely access content based on its actual structure
                if isinstance(thread_message.content, list) and thread_message.content:
                    first_content = thread_message.content[0]
                    if hasattr(first_content, 'text') and hasattr(first_content.text, 'value'):
                        content = first_content.text.value
                    else:
                        content = str(first_content)  # Fallback to string representation
                else:
                    content = str(thread_message.content)  # Handle non-list or empty content

                # Printing the `role` makes the result conversational for users
                print(f"{role.capitalize()}: {content}")
            
            break
        elif run.status == "failed":
            messages = openai_client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt_message}\nAssistant:\n{answer}\n")
            # Handle failed
            break

        
        else:
            time.sleep(5)

print("Function `process_message` created successfully.")

In [ ]:
"""
import time 

def get_assistant_response(thread_id, run_id, user_prompt):
   
    Interacts with the assistant by sending a user prompt, handling function calls if required, 
    and retrieving the conversation messages as a response.

    Parameters:
        thread_id (str): ID of the conversation thread.
        run_id (str): ID of the current assistant run.
        user_prompt (str): The user's prompt or question.

    Returns:
        list of dict: Each message as a dictionary with 'role' and 'content' keys.
    
    # Send initial prompt - Add a message to the thread with the role of "user" and the provided content  
    openai_client.beta.threads.messages.create(
        thread_id=thread_id,    # Use the ID of the thread passed through to the function
        content=user_prompt,    # User content message
        role="user"             # Role of the message sender
    )

    # Polling to monitor Run status
    while True:
        # Retrieve and inspect the assistant's current run status
        run = openai_client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)

        if run.status == "failed":
            print("Assistant run failed. Please try again.")
            break
            #return [{"role": "system", "content": "Assistant run failed. Please try again."}]
        
        elif run.status == "completed":
            # Collect messages if run is complete
            messages = openai_client.beta.threads.messages.list(thread_id=thread_id, order="asc")
            #conversation = [{"role": msg.role, "content": msg.content[0].text.value} for msg in messages.data]

            # Loop through messages and print content based on role
            for msg in messages.data:
                role = msg.role
                content = msg.content[0].text.value
                print(f"{role.capitalize()}: {content}")
            break

            #for msg in conversation:  # Display the conversation
            #    print(f"{msg['role'].capitalize()}: {msg['content']}")
            #return conversation
        
        elif run.status == "requires_action" and run.required_action.type == "submit_tool_outputs":
            print("Function calling initiated...")
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            tool_responses = []
            
            # Iterate over each function call requested by the assistant
            for call in tool_calls:
                # Check if the call is a function and if the function exists is our custom function
                if call.type == "function" and call.function.name == "fetch_stock_price":
                    func = available_functions[call.function.name]  # Retrieve the function reference
                    
                    # Parse the function arguments from JSON and execute the function
                    tool_response = func(**json.loads(call.function.arguments))
                    
                    # Store the tool call ID and output to later send back to the assistant
                    tool_responses.append({"tool_call_id": call.id, "output": tool_response})
                    print(f"Executed '{call.function.name}'. Output: {tool_response}")
                
                else:
                    # Raise an error if the function is not in available_functions to handle unexpected requests
                    raise ValueError(f"Requested function '{call.function.name}' is not available.")
            
            # Submit all collected tool outputs back to the assistant to satisfy the required action
            run = openai_client.beta.threads.runs.submit_tool_outputs(
                thread_id=thread_id, 
                run_id=run_id, 
                tool_outputs=tool_responses
            )
            print("Function call(s) completed successfully.")
        
        else:
            # If the assistant is still processing, wait before checking the status again
            print("Waiting for the Assistant to process...")
            time.sleep(5)
"""

In [ ]:
user_prompt = "What is the current stock price for Amazon?"

response = get_assistant_response(thread.id, run.id, user_prompt)
#for message in response:
#    print(f"{message['role'].capitalize()}: {message['content']}")

#### More Examples using Function
- one basic with image (uses CI)
- one basic function calling 

### Wrap Up

## **Cleanup Resources**
To avoid creating redundant resources and ensure a clean environment, this cell deletes the assistant, thread, and any other created resources. Run this cell at the end of your session to clean up.


In [ ]:
#response = openai_client.beta.assistants.delete(assistant.id)
#print(response)

# Optionally delete any other temporary files or datas
# Note: Any uploaded files to OpenAI could also be cleaned up if needed
